## Librerias

In [0]:
from keras.models import Model
from keras.layers import Activation, Input, concatenate, Dropout, Dense, Flatten
from keras.layers import MaxPooling2D, Conv2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

import os
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import glob
import matplotlib
#import itertools
#import shutil
import IPython
%matplotlib inline


### Conectar con google drive


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Lista de imagenes

In [20]:
#data
PATH="/content/drive/My Drive/topicos en ia" #la carpeta esta compartida
#DATASET_DIR=PATH+"/covid19DatasetRxAPActualizada"
#DATASET_DIR=PATH+"/covid-19-x-ray-10000-images/dataset"
DATASET_DIR=PATH+"/datasetOriginal1000"
print(os.listdir(DATASET_DIR))

normal=[]
for i in glob.glob(DATASET_DIR+"/normal/*"):
  normal.append(matplotlib.image.imread(i))

covid=[]
for i in glob.glob(DATASET_DIR+"/covid/*"):
  covid.append(matplotlib.image.imread(i))

print(len(covid))
print(len(normal))

['covid', 'normal']
70
28


## DATA AUGMENTATION
![texto alternativo](https://www.pyimagesearch.com/wp-content/uploads/2019/07/keras_data_augmentation_in_place.png)

In [21]:
IMG_W = 299
IMG_H = 299
CHANNELS = 3

INPUT_SHAPE = (IMG_W, IMG_H, CHANNELS)
NB_CLASSES = 2
EPOCHS = 41
BATCH_SIZE = 6

# data augmentation
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        rescale = 1./255,
        validation_split = 0.3) 
#division
train_generator = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_H, IMG_W),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training')
validation_generator = datagen.flow_from_directory(
    DATASET_DIR, 
    target_size=(IMG_H, IMG_W),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle= False,
    subset='validation')

Found 69 images belonging to 2 classes.
Found 29 images belonging to 2 classes.


## Block Stem

<a href="https://ibb.co/JHfqL8G"><img src="https://i.ibb.co/JHfqL8G/stem.png" alt="stem" border="0"></a> 

In [0]:
def conv2d(net, filters, kernel_size, strides=(1, 1), padding='same'):
    net = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, use_bias=False)(net)
    net = BatchNormalization()(net)
    net = Activation('relu')(net)
    return net

def blockStem(inputs):
    net = conv2d(inputs, 32, (3, 3), strides=(2, 2), padding='valid')
    net = conv2d(net, 32, (3, 3), padding='valid')
    net = conv2d(net, 64, (3, 3))

    rama1 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(net)
    rama2 = conv2d(net, 96, (3, 3), strides=(2, 2), padding='valid')

    net = concatenate([rama1, rama2])

    rama1 = conv2d(net, 64, (1, 1))
    rama1 = conv2d(rama1, 96, (3, 3), padding='valid')

    rama2 = conv2d(net, 64, (1, 1))
    rama2 = conv2d(rama2, 64, (1, 7))
    rama2 = conv2d(rama2, 64, (7, 1))
    rama2 = conv2d(rama2, 96, (3, 3), padding='valid')

    net = concatenate([rama1, rama2])

    rama1 = conv2d(net, 192, (3, 3), strides=(2, 2), padding='valid')  # different from the paper
    rama2 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(net)

    net = concatenate([rama1, rama2])

    return net

## Block Inception a


 <a href="https://ibb.co/GWMQ7c3"><img src="https://i.ibb.co/GWMQ7c3/ina.png" alt="ina" border="0"></a>


In [0]:
def blockInceptionA(inputs):
    rama1 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    rama1 = conv2d(rama1, 96, (1, 1))


    rama2 = conv2d(inputs, 96, (1, 1))

    rama3 = conv2d(inputs, 64, (1, 1))
    rama3 = conv2d(rama3, 96, (3, 3))

    rama4 = conv2d(inputs, 64, (1, 1))
    rama4 = conv2d(rama4, 96, (3, 3))
    rama4 = conv2d(rama4, 96, (3, 3))

    return concatenate([rama1, rama2, rama3, rama4])

## Block Reduction A

<a href="https://ibb.co/8KH35ZG"><img src="https://i.ibb.co/8KH35ZG/reda.png" alt="reda" border="0"></a>


In [0]:
def blockReductionA(inputs):
    rama1 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(inputs)

    rama2 = conv2d(inputs, 384, (3, 3), strides=(2, 2), padding='valid')

    rama3 = conv2d(inputs, 192, (1, 1))
    rama3 = conv2d(rama3, 224, (3, 3))
    rama3 = conv2d(rama3, 256, (3, 3), strides=(2, 2), padding='valid')

    return concatenate([rama1, rama2, rama3])

## Block inception B

<a href="https://ibb.co/N77WfPQ"><img src="https://i.ibb.co/N77WfPQ/inb.png" alt="inb" border="0"></a>


In [0]:

def blockInceptionB(inputs):
    rama1 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    rama1 = conv2d(rama1, 128, (1, 1))

    rama2 = conv2d(inputs, 384, (1, 1))

    rama3 = conv2d(inputs, 192, (1, 1))
    rama3 = conv2d(rama3, 224, (1, 7))
    rama3 = conv2d(rama3, 256, (7, 1))  # different from the paper

    rama4 = conv2d(inputs, 192, (1, 1))
    rama4 = conv2d(rama4, 192, (1, 7))
    rama4 = conv2d(rama4, 224, (7, 1))
    rama4 = conv2d(rama4, 224, (1, 7))
    rama4 = conv2d(rama4, 256, (7, 1))

    return concatenate([rama1, rama2, rama3, rama4])

## Block Reduction B

<a href="https://ibb.co/k5cz080"><img src="https://i.ibb.co/k5cz080/redb.png" alt="redb" border="0" width="100"></a> 


In [0]:
def blockReductionB(inputs):
    rama1 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(inputs)

    rama2 = conv2d(inputs, 192, (1, 1))
    rama2 = conv2d(rama2, 192, (3, 3), strides=(2, 2), padding='valid')

    rama3 = conv2d(inputs, 256, (1, 1))
    rama3 = conv2d(rama3, 256, (1, 7))
    rama3 = conv2d(rama3, 320, (7, 1))
    rama3 = conv2d(rama3, 320, (3, 3), strides=(2, 2), padding='valid')

    return concatenate([rama1, rama2, rama3])


## Block Inception C

<a href="https://ibb.co/dKqHVN9"><img src="https://i.ibb.co/dKqHVN9/inc.png" alt="inc" border="0"></a> 



In [0]:
def blockInceptionC(inputs):
    rama1 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    rama1 = conv2d(rama1, 256, (1, 1))

    rama2 = conv2d(inputs, 256, (1, 1))

    rama3 = conv2d(inputs, 384, (1, 1))
    rama3_1 = conv2d(rama3, 256, (1, 3))
    rama3_2 = conv2d(rama3, 256, (3, 1))

    rama4 = conv2d(inputs, 384, (1, 1))
    rama4 = conv2d(rama4, 512, (3, 1))
    rama4 = conv2d(rama4, 448, (1, 3))
    rama4_2 = conv2d(rama4, 256, (1, 3))
    rama4_1 = conv2d(rama4, 256, (3, 1))

    return concatenate([rama1, rama2, rama3_1, rama3_2, rama4_1, rama4_2])


![texto alternativo](https://miro.medium.com/max/2000/1*15sIUNOxqVyDPmGGqefyVw.png)





In [0]:
def inceptionV4(classes_num=1, image_height=299, image_width=299, image_channel=3):
    inputs = Input((image_height, image_width, image_channel))

    # 299 x 299 x 3
    net = blockStem(inputs)

    # 4 x Inception-A ( Output: 35 x 35 x 384 )
    for i in range(4):
        net = blockInceptionA(net)

    # Reduction-A ( Output: 17 x 17 x 1024 )
    net = blockReductionA(net)

    # 7 x Inception-B ( Output: 17 x 17 x 1024 )
    for i in range(7):
        net = blockInceptionB(net)

    # Reduction-B ( Output: 8 x 8 x 1536 )
    net = blockReductionB(net)

    # 3 x Inception-C ( Output: 8 x 8 x 1536 )
    for i in range(3):
        net = blockInceptionC(net)

    # Average Pooling ( Output: 1536 )
    net = AveragePooling2D((8, 8))(net)

    # Dropout ( keep 0.8 )
    net = Dropout(0.3)(net)
    net = Flatten()(net)

    # Output
    outputs = Dense(units=1, activation='sigmoid')(net)

    return Model(inputs, outputs, name='Inception-v4')

# Create model

In [0]:
model = inceptionV4()

# Model summary

In [30]:
model.summary()

Model: "Inception-v4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_150 (Conv2D)             (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_150 (BatchN (None, 149, 149, 32) 128         conv2d_150[0][0]                 
__________________________________________________________________________________________________
activation_150 (Activation)     (None, 149, 149, 32) 0           batch_normalization_150[0][0]    
_______________________________________________________________________________________

In [0]:
import tensorflow as tf

model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss='binary_crossentropy',
              metrics=['accuracy'])
#model.fit(train_generator,validation_data=validation_generator,batch_size=16,epochs=EPOCHS,verbose=1)

In [0]:
history=model.fit(train_generator,validation_data=validation_generator,epochs=EPOCHS)
"""
history=model.fit_generator(train_generator,
                            steps_per_epoch=train_generator.samples//BATCH_SIZE,
                            validation_data=validation_generator,
                            validation_steps=validation_generator.samples//BATCH_SIZE,
                            epochs=EPOCHS)
"""


In [0]:
#metricas
#acc
print("training_accuracy", history.history['accuracy'][-1])
print("validation_accuracy", history.history['val_accuracy'][-1])

#plot
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#matriz de confusion
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import seaborn as sns

true_class = validation_generator.classes
pred_class = model.predict(validation_generator)
pred_class = np.around(pred_class , decimals = 0)
cm = confusion_matrix(true_class,pred_class)
print(cm)
cm = pd.DataFrame(cm , index = ['0','1'] , columns = ['0','1'])
plt.figure(figsize = (10,10))
sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , annot = True, fmt='')

## Guardar Modelo

In [0]:
# guardar modelo
model.save_weights("model1.h5")
print("Guardado)

# Cargar Modelo

In [0]:
#cargar modelo
#loaded_model.load_weights("model1.h5")
#print("Cargado")

## Clasificar

In [0]:
model_path="/content/drive/My Drive/topicos en ia/model11.h5"

model.load_weights(model_path)

In [0]:
from keras.models import load_model
from keras.preprocessing import image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from PIL import Image

import numpy as np
import os
#input es la imagen de rayosx que dara el profe para clasificar, esto deberia ir

folder_path="/content/drive/My Drive/topicos en ia/profe/in"

IMG_W=299
IMG_H=299

img=load_img(folder_path+"/normal1.jpeg",target_size=(IMG_W,IMG_H))
img=img_to_array(img)
img=np.expand_dims(img, axis=0) #primera dimesion es indice de batch

p=model.predict(img)
print(p)

In [0]:
for c in correct[:6]:
    plt.subplot(3,2,i+1)
    plt.imshow(validation_generator[0][0][c].reshape(150,150,3))
    plt.title("Predicted Class {},Actual Class {}".format(pred_class.reshape(1,-1)[0][c], true_class[c]))
    plt.tight_layout()
    i += 1

In [0]:
#ESTO FUE PARA ACTUALIZAR EL DATASET AL INICIO

import pandas as pd
import os
import shutil

file_path = "covid-chestxray-dataset/metadata.csv"
images_path="covid-chestxray-dataset/images"

df=pd.read_csv(file_path)
print(df.shape)

target_dir="covid"
cnt=0
for (i,row) in df.iterrows():
    if row["finding"]=="COVID-19" and row["view"]=="AP":
        filename = row["filename"]
        image_path=os.path.join(images_path, filename)
        image_copy_path=os.path.join(target_dir,filename)
        shutil.copy2(image_path,image_copy_path)
        cnt+=1
        
print(cnt)

import random
path="chest_xray_pneumonia/train/NORMAL"
target= "normal"
names=os.listdir(path)
random.shuffle(names)
cnt2=0
for i in range(cnt):
    name=names[i]
    image_path=os.path.join(path,name)
    target_=os.path.join(target,name)
    shutil.copy2(image_path,target_)
    cnt2+=1
print(cnt2)